## Dataframe Cleaning/Merging

The objective of this notebook is to clean the notebooks of missing values, and impute additional features which I can calculate from the box score. These features I think will be useful for capturing additional variance in my data. In addition, I merge the game data with the corresponding gambling lines in this notebook.

In [1]:
import json
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd
import matplotlib.pyplot as plt
import re

%matplotlib inline

In [2]:
team_handles_dict = {'Toronto Raptors': 'TOR',
                     'Boston Celtics': 'BOS',
                     'Philadelphia 76ers': 'PHI',
                     'Cleveland Cavaliers': 'CLE',
                     'Indiana Pacers': 'IND',
                     'Miami Heat': 'MIA',
                     'Milwaukee Bucks': 'MIL',
                     'Washington Wizards': 'WAS',
                     'Detroit Pistons': 'DET',
                     #'Charlotte Hornets': 'CHO',
                     #'Charlotte Bobcats': 'CHA',
                     'New York Knicks': 'NYK',
                     'Brooklyn Nets': 'BRK',
                     'Chicago Bulls': 'CHI',
                     'Orlando Magic': 'ORL',
                     'Atlanta Hawks': 'ATL',
                     'Houston Rockets': 'HOU',
                     'Golden State Warriors': 'GSW',
                     'Portland Trail Blazers': 'POR',
                     'Oklahoma City Thunder': 'OKC',
                     'Utah Jazz': 'UTA',
                     'New Orleans Pelicans': 'NOP',
                     'San Antonio Spurs': 'SAS',
                     'Minnesota Timberwolves': 'MIN',
                     'Denver Nuggets': 'DEN',
                     'L.A. Clippers': 'LAC',
                     'L.A. Lakers': 'LAL',
                     'Sacramento Kings': 'SAC',
                     'Dallas Mavericks': 'DAL',
                     'Memphis Grizzlies': 'MEM',
                     'Phoenix Suns': 'PHO'}

In [3]:
def dataframe_loader(years_games):
    years_stats = []
    for game in years_games:
        with open(f'{game}') as g:
            years_stats.append(json.load(g))
    all_games_year = [team for game_list in years_stats for game in game_list for team in game]
    df_year = pd.DataFrame(all_games_year, columns=['gid', 'team_slug', 'away_home', 'mp', 'fg', 'fga',
                                                    'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb',
                                                    'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts'])
    return df_year

In [4]:
gl_2014 = !ls ../*_2014.json
gl_2015 = !ls ../*_2015.json
gl_2016 = !ls ../*_2016.json
gl_2017 = !ls ../*_2017.json
gl_2018 = !ls ../*_2018.json

df_2014 = dataframe_loader(gl_2014)
df_2015 = dataframe_loader(gl_2015)
df_2016 = dataframe_loader(gl_2016)
df_2017 = dataframe_loader(gl_2017)
df_2018 = dataframe_loader(gl_2018)

In [5]:
df_all = df_2014.append([df_2015, df_2016, df_2017, df_2018], ignore_index=True)

In [6]:
df_all['date'] = df_all['gid'].map(lambda x: x[:8])

In [7]:
df_all.head()

gid team_slug away_home   mp  fg fga   fg%  3p 3pa   3p%  \
0  201311010ATL       TOR      away  240  40  88  .455   7  23  .304   
1  201311010ATL       ATL      home  240  36  77  .468  10  23  .435   
2  201311090ATL       ORL      away  240  38  82  .463   8  17  .471   
3  201311090ATL       ATL      home  240  43  90  .478   6  19  .316   
4  201311130ATL       NYK      away  240  37  88  .420  12  34  .353   

     ...    orb drb trb ast stl blk tov  pf  pts      date  
0    ...     10  32  42  15   5   6  12  25   95  20131101  
1    ...      7  30  37  24   7   3  13  12  102  20131101  
2    ...      9  27  36  22   9   2  17  24   94  20131109  
3    ...      7  32  39  36   9   6  12  15  104  20131109  
4    ...      8  29  37  24  11   3   3  19   95  20131113  

[5 rows x 23 columns]

In [8]:
with open('../all_gambling_lines.json') as g:
     all_lines = json.load(g)

In [9]:
df_lines = pd.DataFrame(data=all_lines[0])
for day_line in all_lines[1:]:
    df_lines = df_lines.append(day_line, ignore_index=True).copy()

In [10]:
df_lines.head()

0             1          2
0  20131101     Cleveland   -3½ -105
1  20131101     Charlotte   188 -110
2  20131101   New Orleans    -4 -105
3  20131101       Orlando  193½ -107
4  20131101  Philadelphia   201 -105

Writing this cell to make corrections to the lines for data points where I have to manually impute the line because the line was missing from the website I scraped

In [11]:
df_lines.loc[384, 2] = '-1 -105'
df_lines.loc[385, 2] = '194.5 -105'
df_lines.loc[2466, 2] = '203.5 -105'
df_lines.loc[2467, 2] = '-1 -105'
df_lines.loc[5677, 2] = '0 -105'

Writing this cell to append missing lines to my Dataframe

In [12]:
missing_lines = pd.DataFrame([['20150306', 'Miami', '+6 -110'], 
                             ['20150306', 'Washington','193 -110']])

In [13]:
df_lines = df_lines.append(missing_lines, ignore_index=True)

In [14]:
df_lines.rename({0:'date', 1:'team', 2:'full_line'}, axis=1, inplace=True)

In [15]:
df_lines['team_slug'] = df_lines['team']

I wrote this cell to get the team_slug from a team name. With Charlotte, because there is a split between the slugs based on time, I set all of the 2014 season's games to the Charlotte Bobcats slug, and every seasons' games following to have the clug of the Charlotte Hornets

In [16]:
for i, team in enumerate(df_lines['team']):
    if team == "Charlotte":
        if int(df_lines['date'][i]) < 20141001:
            df_lines['team_slug'][i] = 'CHA'
        else:
            df_lines['team_slug'][i] = 'CHO'
    else:
        for key in team_handles_dict:     
            if team in key:
                df_lines['team_slug'][i] = team_handles_dict[key]
            

In [17]:
df_lines['gid'] = df_lines['date'] + '0' + df_lines['team_slug']

In [18]:
for i, entry in enumerate(df_lines['gid']):
    if i % 2 == 0:
        df_lines['gid'][i] = df_lines['gid'][i+1]

In the cell below I merge my game data and my gambling lines data, using both team slug and game ID as the features to join on

In [19]:
df = pd.merge(df_all, df_lines, how='inner', on=['gid', 'team_slug'])

In [20]:
df.head()

gid team_slug away_home   mp  fg fga   fg%  3p 3pa   3p%  \
0  201311010ATL       TOR      away  240  40  88  .455   7  23  .304   
1  201311010ATL       ATL      home  240  36  77  .468  10  23  .435   
2  201311090ATL       ORL      away  240  38  82  .463   8  17  .471   
3  201311090ATL       ATL      home  240  43  90  .478   6  19  .316   
4  201311130ATL       NYK      away  240  37  88  .420  12  34  .353   

     ...     ast stl blk tov  pf  pts    date_x    date_y      team  full_line  
0    ...      15   5   6  12  25   95  20131101  20131101   Toronto   195 -113  
1    ...      24   7   3  13  12  102  20131101  20131101   Atlanta   -3½ -104  
2    ...      22   9   2  17  24   94  20131109  20131109   Orlando   199 -108  
3    ...      36   9   6  12  15  104  20131109  20131109   Atlanta   -6½ -109  
4    ...      24  11   3   3  19   95  20131113  20131113  New York  201½ -105  

[5 rows x 26 columns]

In [21]:
len(df)

12300

In [22]:
home_away_dict = {'away': 0, 'home': 1}

df['home'] = df.away_home.map(lambda x: home_away_dict[x]).copy()

In [23]:
df['betting_line'] = df['full_line'].map(lambda x: str.split(x)[0])
df['bet_terms'] = df['full_line'].map(lambda x: str.split(x)[1])

The following cells are Regex equations I made to remove the 1/2 symbol from the betting lines, and impute .5 in its place. I did this because the 1/2 symbol was recognized as a special character and not a number

In [24]:
p = re.compile(r'[^0-9+-]+')

In [25]:
for i, entry in enumerate(df['betting_line']):
    if p.findall(entry):
        df.loc[i, 'betting_line'] = entry.replace(p.findall(entry)[0], '.5')

In [26]:
df['date'] = pd.to_datetime([x[0:4] + '-' + x[4:6] + '-' + x[6:8] for x in df['date_x']])

In [27]:
df.drop(['date_x', 'team', 'mp', 'away_home', 'full_line', 'date_y'], axis=1, inplace=True)

In [28]:
df.sort_values(by=['team_slug', 'date'], inplace=True)

In [29]:
df[['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
           'orb', 'drb', 'trb', 'ast', 'stl', 
           'blk', 'tov', 'pf', 'pts']] = df[['fg', 'fga', '3p', '3pa', 'ft', 'fta', 
           'orb', 'drb', 'trb', 'ast', 'stl', 
           'blk', 'tov', 'pf', 'pts']].astype('int64', copy=True)

df[['fg%', '3p%', 'ft%', 
    'betting_line', 'bet_terms']] = df[['fg%', '3p%', 
                                        'ft%', 'betting_line', 'bet_terms']].astype('float64', copy=True)

By merging the dataframe on itself and then removing rows where the merge imputed the same information twice on one row, I am able to return a dataframe which represents a game on one row, as both the target team's box score and their opponent's box score. Every game is represented twice, once for each team participating in the game. 

In [30]:
doubled_df = df.merge(df, on='gid',  suffixes=['_1', '_2'])

In [31]:
merged_df = doubled_df[doubled_df['team_slug_1'] != doubled_df['team_slug_2']].copy()

In [32]:
merged_df.head()

gid team_slug_1  fg_1  fga_1  fg%_1  3p_1  3pa_1  3p%_1  ft_1  \
1  201310300DAL         ATL    37     76  0.487     8     24  0.333    27   
2  201310300DAL         DAL    44     77  0.571    11     24  0.458    19   
5  201311010ATL         ATL    36     77  0.468    10     23  0.435    20   
6  201311010ATL         TOR    40     88  0.455     7     23  0.304     8   
9  201311030LAL         ATL    40     90  0.444     9     22  0.409    14   

   fta_1    ...      ast_2  stl_2  blk_2  tov_2  pf_2  pts_2  home_2  \
1     35    ...         31      7      2     20    22    118       1   
2     24    ...         27     16      5     17    20    109       0   
5     31    ...         15      5      6     12    25     95       0   
6     14    ...         24      7      3     13    12    102       1   
9     26    ...         27      5      7     13    24    105       1   

   betting_line_2  bet_terms_2     date_2  
1            -6.0       -106.0 2013-10-30  
2           195.5       -105.0 2013-10-30  
5           195.0       -113.0 2013-11-01  
6            -3.5       -104.0 2013-11-01  
9            -1.0       -102.0 2013-11-03  

[5 rows x 47 columns]

Equation for Offensive Rating: 

Offensive Rating = 100 x Pts / (0.5 * ((Tm FGA + 0.4 * Tm FTA - 1.07 * (Tm ORB / (Tm ORB + Opp DRB)) * (Tm FGA - Tm FG) + Tm TOV) + (Opp FGA + 0.4 * Opp FTA - 1.07 * (Opp ORB / (Opp ORB + Tm DRB)) * (Opp FGA - Opp FG) + Opp TOV)))

In [33]:
merged_df['off_rating_1'] = merged_df.apply((lambda x: 100 * x['pts_1'] / 
                (0.5*((x['fga_1'] + 0.4*(x['fta_1']) - 1.07*(x['orb_1'] / (x['orb_1'] + x['drb_2']))
                 * (x['fga_1'] - x['fg_1']) + x['tov_1']) +
                (x['fga_2'] + 0.4*(x['fta_2']) - 1.07*(x['orb_2'] / (x['orb_2'] + x['drb_1']))
                 * (x['fga_2'] - x['fg_2']) + x['tov_2'])))), 1)

In [34]:
merged_df['off_rating_2'] = merged_df.apply((lambda x: 100 * x['pts_2'] / 
                (0.5*((x['fga_2'] + 0.4*(x['fta_2']) - 1.07*(x['orb_2'] / (x['orb_2'] + x['drb_1']))
                 * (x['fga_2'] - x['fg_2']) + x['tov_2']) +
                (x['fga_1'] + 0.4*(x['fta_1']) - 1.07*(x['orb_1'] / (x['orb_1'] + x['drb_2']))
                 * (x['fga_1'] - x['fg_1']) + x['tov_1'])))), 1)

In [35]:
merged_df['off_rating_1'] = round(merged_df['off_rating_1'], 2)
merged_df['off_rating_2'] = round(merged_df['off_rating_2'], 2)

In [36]:
merged_df.reset_index(inplace=True)

In [37]:
over_under_list = []
for i, x in enumerate(merged_df['betting_line_1']):
    if x > merged_df['betting_line_2'][i]:
        over_under_list.append(x)
    else:
        over_under_list.append(merged_df['betting_line_2'][i])

In [38]:
merged_df['over_under'] = pd.Series(over_under_list, merged_df.index)

In [39]:
for i, value in enumerate(merged_df['betting_line_1']):
    if merged_df.loc[i, 'betting_line_1'] > 0:
        merged_df.loc[i, 'betting_line_1'] = merged_df.loc[i, 'betting_line_2'] * -1
    else:
        merged_df.loc[i, 'betting_line_2'] = merged_df.loc[i, 'betting_line_1'] * -1
        

In [40]:
merged_df.set_index('date_2', inplace=True)

In [41]:
merged_df.drop(['bet_terms_1', 'bet_terms_2', 'date_1', 'index'], axis=1, inplace=True)

In [42]:
merged_df.head()

gid team_slug_1  fg_1  fga_1  fg%_1  3p_1  3pa_1  3p%_1  \
date_2                                                                         
2013-10-30  201310300DAL         ATL    37     76  0.487     8     24  0.333   
2013-10-30  201310300DAL         DAL    44     77  0.571    11     24  0.458   
2013-11-01  201311010ATL         ATL    36     77  0.468    10     23  0.435   
2013-11-01  201311010ATL         TOR    40     88  0.455     7     23  0.304   
2013-11-03  201311030LAL         ATL    40     90  0.444     9     22  0.409   

            ft_1  fta_1     ...      stl_2  blk_2  tov_2  pf_2  pts_2  home_2  \
date_2                      ...                                                 
2013-10-30    27     35     ...          7      2     20    22    118       1   
2013-10-30    19     24     ...         16      5     17    20    109       0   
2013-11-01    20     31     ...          5      6     12    25     95       0   
2013-11-01     8     14     ...          7      3     13    12    102       1   
2013-11-03    14     26     ...          5      7     13    24    105       1   

            betting_line_2  off_rating_1  off_rating_2  over_under  
date_2                                                              
2013-10-30            -6.0        108.28        117.23       195.5  
2013-10-30             6.0        117.23        108.28       195.5  
2013-11-01             3.5        108.92        101.45       195.0  
2013-11-01            -3.5        101.45        108.92       195.0  
2013-11-03            -1.0         99.62        101.55       206.5  

[5 rows x 46 columns]

In [43]:
merged_df.to_csv('clean_nba_betting_dataframe_full.csv', columns=merged_df.columns)